In [ ]:
# bootstrap
from scipy.stats import bootstrap

def boots(x, y): # x = climatology, y = composite
    
    ncols = len(x.lon)
    nrows = len(x.lat)
    #print(ncols, nrows)
    
    boots = xr.DataArray(data=np.zeros((nrows,ncols)), dims=["lat","lon"], coords=[x.lat, x.lon])
    for j in range(nrows):
        for k in range(ncols):
            data = x.isel(lat=j,lon=k).values
            #print(data)
            if np.any(np.isnan(data)) or np.all((data==0)) or all(element == data[0] for element in data):
                boots[j,k] = 1
            else:
                ci_l1, ci_u1 = bootstrap((data,), np.mean, confidence_level=0.9,
                                         n_resamples=1000).confidence_interval
                
                data2 = y.isel(lat=j,lon=k).values
                if ci_l1 < data2.mean() < ci_u1: #((data2.mean() > ci_l1) and (data2.mean() < ci_u1)):
                    boots[j,k] = 0
                else:
                    boots[j,k] = 1 #use this for significance
                
    return boots